In [3]:
%pip install -q "langchain>=0.1.0" "langchain-experimental>=0.0.45" "langchain-openai>=0.1.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [14]:
%pip install -q langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.6/55.6 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 473.0/473.0 kB 29.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-experimental 0.3.4 requires langchain-core<0.4.0,>=0.3.28, but you have langchain-core 1.0.7 which is incompatible.
langchain-openai 0.3.35 requires langchain-core<1.0.0,>=0.3.78, but you have langchain-core 1.0.7 which is incompatible.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.9.0 which is incompatible.
langchain 0.3.27 requires langchain-core<1.0.0,>=0.3.72, but you have langchain-core 1.0.7 which is incompatible.


In [16]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_core.documents import Document
from langchain_openai import ChatOpenAI

In [5]:
from google.colab import userdata

In [18]:
llm = ChatOpenAI(
    temperature=0, model_name="gpt-4o-mini", api_key=userdata.get("OPENAI_APIKEY")
  )

graph_transformer = LLMGraphTransformer(llm=llm)

In [7]:
%pip install -q pymupdf4llm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 83.7 MB/s eta 0:00:00


In [19]:
from pathlib import Path
from tqdm import tqdm

import pymupdf4llm

LECTURES_DIR = Path("/content/lectures")

texts = []
for file in tqdm(LECTURES_DIR.iterdir()):
  md_text = pymupdf4llm.to_markdown(file)
  texts.append(md_text)

3it [00:02,  1.08it/s]


In [20]:
documents = [Document(page_content=text) for text in texts]

graph_documents = graph_transformer.convert_to_graph_documents(documents)

In [21]:
print(f"Nodes:{graph_documents[0].nodes}")
print(f"Relationships:{graph_documents[0].relationships}")

Nodes:[Node(id='Биполярный Транзистор', type='Транзистор', properties={}), Node(id='Эмиттер', type='Электрод', properties={}), Node(id='Коллектор', type='Электрод', properties={}), Node(id='База', type='Электрод', properties={}), Node(id='Источник Еэ', type='Источник', properties={}), Node(id='Источник Ек', type='Источник', properties={}), Node(id='Iэ', type='Ток', properties={}), Node(id='Iк', type='Ток', properties={}), Node(id='Iб', type='Ток', properties={}), Node(id='Rк', type='Сопротивление', properties={}), Node(id='Активный Режим', type='Режим работы', properties={}), Node(id='Инверсный Режим', type='Режим работы', properties={}), Node(id='Режим Насыщения', type='Режим работы', properties={}), Node(id='Режим Отсечки', type='Режим работы', properties={}), Node(id='Динамический Режим', type='Режим работы', properties={}), Node(id='Статический Режим', type='Режим работы', properties={}), Node(id='Проводимость', type='Свойство', properties={}), Node(id='Коэффициент Передачи По Току

In [22]:
%pip install -q "pyvis>=0.3.2"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 50.3 MB/s eta 0:00:00


In [23]:
import os

from pyvis.network import Network

def visualize_graph(graph_documents):

    # Create network
    net = Network(height="1200px", width="100%", directed=True,
                      notebook=False, bgcolor="#222222", font_color="white")

    nodes = graph_documents[0].nodes
    relationships = graph_documents[0].relationships

    # Build lookup for valid nodes
    node_dict = {node.id: node for node in nodes}

    # Filter out invalid edges and collect valid node IDs
    valid_edges = []
    valid_node_ids = set()
    for rel in relationships:
        if rel.source.id in node_dict and rel.target.id in node_dict:
            valid_edges.append(rel)
            valid_node_ids.update([rel.source.id, rel.target.id])


    # Track which nodes are part of any relationship
    connected_node_ids = set()
    for rel in relationships:
        connected_node_ids.add(rel.source.id)
        connected_node_ids.add(rel.target.id)

    # Add valid nodes
    for node_id in valid_node_ids:
        node = node_dict[node_id]
        try:
            net.add_node(node.id, label=node.id, title=node.type, group=node.type)
        except:
            continue  # skip if error

    # Add valid edges
    for rel in valid_edges:
        try:
            net.add_edge(rel.source.id, rel.target.id, label=rel.type.lower())
        except:
            continue  # skip if error

    # Configure physics
    net.set_options("""
            {
                "physics": {
                    "forceAtlas2Based": {
                        "gravitationalConstant": -100,
                        "centralGravity": 0.01,
                        "springLength": 200,
                        "springConstant": 0.08
                    },
                    "minVelocity": 0.75,
                    "solver": "forceAtlas2Based"
                }
            }
            """)

    output_file = "knowledge_graph.html"
    net.save_graph(output_file)
    print(f"Graph saved to {os.path.abspath(output_file)}")

    # Try to open in browser
    try:
        import webbrowser
        webbrowser.open(f"file://{os.path.abspath(output_file)}")
    except:
        print("Could not open browser automatically")

# Run the function
visualize_graph(graph_documents)

Graph saved to /content/knowledge_graph.html


In [24]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [25]:
CONSPECT_PROMPT = """
На основе следующего графа знаний создай структурированный конспект лекции:

УЗЛЫ ГРАФА:
{nodes}

СВЯЗИ ГРАФА:
{relationships}

Создай подробный конспект лекции со следующей структурой:
1. Основные понятия и определения
2. Ключевые компоненты и их характеристики
3. Взаимосвязи между компонентами
4. Классификации и режимы работы
5. Практические аспекты применения
6. Объясни сложные концепции, на простых примерах

Конспект должен быть логичным, последовательным и образовательным.
Ориентируйся на студентов технических специальностей.

КОНСПЕКТ В ФОРМАТЕ MARKDOWN:
"""

In [26]:
def extract_graph_info(graph_docs):
  """Извлекает информацию о узлах и связях из графа"""
  nodes_info = []
  relationships_info = []

  for graph_doc in graph_docs:
      for node in graph_doc.nodes:
          nodes_info.append(f"- {node.id} ({node.type})")

      for rel in graph_doc.relationships:
          relationships_info.append(f"- {rel.source.id} --{rel.type}--> {rel.target.id}")

  return "\n".join(nodes_info), "\n".join(relationships_info)

# Создаем цепочку
conspect_chain = (
    {
        "nodes": lambda x: extract_graph_info(x)[0],
        "relationships": lambda x: extract_graph_info(x)[1]
    }
    | ChatPromptTemplate.from_template(CONSPECT_PROMPT)
    | llm
    | StrOutputParser()
)

# Генерируем конспект
conspect = conspect_chain.invoke(graph_documents)

with open("conspect.txt", "w", encoding="utf-8") as file:
  file.write(conspect)

print("КОНСПЕКТ ЛЕКЦИИ:")
print(conspect)

КОНСПЕКТ ЛЕКЦИИ:
# Конспект лекции по биполярным и полевым транзисторам

## 1. Основные понятия и определения
- **Транзистор**: полупроводниковый прибор, используемый для усиления или переключения электрических сигналов.
- **Биполярный транзистор**: транзистор, в котором ток управляется двумя типами носителей заряда (электронами и дырками).
- **Полевой транзистор**: транзистор, в котором ток управляется электрическим полем, создаваемым напряжением на управляющем электроде (затворе).
- **Эмиттер, коллектор, база**: электроды биполярного транзистора, где эмиттер вводит ток, коллектор выводит ток, а база управляет током.
- **Источник Еэ и Ек**: источники, обеспечивающие токи Iэ (эмиттерный ток) и Iк (коллекторный ток) соответственно.

## 2. Ключевые компоненты и их характеристики
- **Электроды**:
  - **Эмиттер**: обеспечивает ввод тока в транзистор.
  - **Коллектор**: выводит ток из транзистора.
  - **База**: управляет током между эмиттером и коллектором.
  
- **Токи**:
  - **Iэ**: эмитте

In [27]:
HIERARCHICAL_CONSPECT_PROMPT = """
Тема лекции: Биполярные транзисторы и их характеристики

Граф знаний:
{graph_summary}

Создай иерархический конспект с тремя уровнями детализации:

1. ОБЗОР (верхний уровень):
   - Основная тема и ключевые концепции
   - Самые важные определения
   - Область применения

2. ДЕТАЛИ (средний уровень):
   - Компоненты и их функции
   - Классификации и режимы работы
   - Основные характеристики и параметры

3. ГЛУБОКОЕ ПОНИМАНИЕ (нижний уровень):
   - Взаимосвязи между компонентами
   - Математические модели и параметры
   - Практические схемы и применения

КОНСПЕКТ:
"""

# Функция для создания сводки графа
def create_graph_summary(graph_docs):
    summary = "УЗЛЫ:\n"
    nodes_by_type = {}

    for graph_doc in graph_docs:
        for node in graph_doc.nodes:
            if node.type not in nodes_by_type:
                nodes_by_type[node.type] = []
            nodes_by_type[node.type].append(node.id)

    for node_type, nodes in nodes_by_type.items():
        summary += f"  {node_type}: {', '.join(nodes)}\n"

    summary += "\nОСНОВНЫЕ СВЯЗИ:\n"
    for graph_doc in graph_docs:
        for rel in graph_doc.relationships[:10]:  # Первые 10 связей
            summary += f"  {rel.source.id} -> {rel.target.id} [{rel.type}]\n"

    return summary

hierarchical_chain = (
    {"graph_summary": lambda x: create_graph_summary(x)}
    | ChatPromptTemplate.from_template(HIERARCHICAL_CONSPECT_PROMPT)
    | llm
    | StrOutputParser()
)

hierarchical_conspect = hierarchical_chain.invoke(graph_documents)

with open("hierarchical_conspect.txt", "w", encoding="utf-8") as file:
  file.write(hierarchical_conspect)

print("ИЕРАРХИЧЕСКИЙ КОНСПЕКТ:")
print(hierarchical_conspect)

ИЕРАРХИЧЕСКИЙ КОНСПЕКТ:
# Конспект по теме: Биполярные транзисторы и их характеристики

## 1. ОБЗОР (верхний уровень)
   - **Основная тема и ключевые концепции**
     - Биполярные транзисторы (БТ) как ключевые элементы в электронике, используемые для усиления и переключения сигналов.
     - Основные электрические явления, связанные с работой БТ, такие как проводимость и управление токами.
   
   - **Самые важные определения**
     - Биполярный транзистор: полупроводниковый прибор, состоящий из трех слоев полупроводникового материала (эмиттер, база, коллектор).
     - Режимы работы: активный, инверсный, насыщения, отсечки и динамический режимы.
   
   - **Область применения**
     - Использование в усилителях, выпрямителях, источниках питания, цифровой электронике и других устройствах.

## 2. ДЕТАЛИ (средний уровень)
   - **Компоненты и их функции**
     - **Эмиттер**: источник носителей заряда (электронов или дырок).
     - **База**: тонкий слой, через который происходит управление ток

In [28]:
# Цепочка для создания конспекта с контрольными вопросами
INTERACTIVE_CONSPECT_PROMPT = """
На основе графа знаний создай образовательный конспект с контрольными вопросами:

ГРАФ ЗНАНИЙ:
{graph_info}

Структура конспекта:

📚 ТЕОРЕТИЧЕСКАЯ ЧАСТЬ:
- Основные понятия и определения
- Принципы работы и функционирования
- Классификации и характеристики

🔧 ПРАКТИЧЕСКАЯ ЧАСТЬ:
- Примеры применения
- Расчетные формулы и параметры
- Типовые схемы включения

❓ КОНТРОЛЬНЫЕ ВОПРОСЫ:
- 5 вопросов для самопроверки по ключевым темам
- Вопросы должны охватывать разные уровни понимания

💡 ВАЖНЫЕ ВЫВОДЫ:
- Основные идеи для запоминания
- Практические рекомендации

КОНСПЕКТ:
"""

interactive_chain = (
    {"graph_info": lambda x: create_graph_summary(x)}
    | ChatPromptTemplate.from_template(INTERACTIVE_CONSPECT_PROMPT)
    | llm
    | StrOutputParser()
)

interactive_conspect = interactive_chain.invoke(graph_documents)
print("ИНТЕРАКТИВНЫЙ КОНСПЕКТ:")

with open("interactive_conspect.txt", "w", encoding="utf-8") as file:
  file.write(interactive_conspect)

print(interactive_conspect)

ИНТЕРАКТИВНЫЙ КОНСПЕКТ:
# Конспект по теме: Транзисторы и их применение в электронике

## 📚 ТЕОРЕТИЧЕСКАЯ ЧАСТЬ:

### Основные понятия и определения
- **Транзистор**: полупроводниковый прибор, используемый для усиления и переключения электрических сигналов. Существует два основных типа: биполярные и полевые транзисторы.
- **Биполярный транзистор**: состоит из трех областей - эмиттера, базы и коллектора. Работает за счет управления током через базу.
- **Полевой транзистор**: управляется электрическим полем, имеет три основных элемента: исток, затвор и сток.

### Принципы работы и функционирования
- **Активный режим**: транзистор работает как усилитель, ток через базу управляет током через коллектор.
- **Режим насыщения**: транзистор полностью открыт, максимальный ток через коллектор.
- **Режим отсечки**: транзистор закрыт, ток через коллектор минимален.
- **Инверсный режим**: транзистор работает в обратном направлении, что может быть использовано в некоторых схемах.

### Классификации и